In [159]:
import pandas as pd
import numpy as np

In [160]:
df=pd.read_csv("blinkit_dynamic_pricing_1000_samples.csv")
df.head()


,Product Name,Base Price,Competitor Price,Demand,Supply,Expiry Days,Optimized Price
0,Kinder Joy,114,114,1,0,16,112
1,Bru Coffee 200g,437,449,0,1,27,429
2,Bournvita 500g,125,126,2,1,18,123
3,Dairy Milk Chocolate,144,148,0,1,15,137
4,Amul Milk 1L,66,65,2,1,12,63


In [161]:
df.isnull().sum()

Product Name        0
Base Price          0
Competitor Price    0
Demand              0
Supply              0
Expiry Days         0
Optimized Price     0
dtype: int64

In [162]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

In [163]:
# encoder = LabelEncoder()
# df["Demand"] = encoder.fit_transform(df["Demand"])  # Convert 'High', 'Medium', 'Low' → 2,1,0
# df["Supply"] = encoder.fit_transform(df["Supply"])

In [164]:
df.head()

,Product Name,Base Price,Competitor Price,Demand,Supply,Expiry Days,Optimized Price
0,Kinder Joy,114,114,1,0,16,112
1,Bru Coffee 200g,437,449,0,1,27,429
2,Bournvita 500g,125,126,2,1,18,123
3,Dairy Milk Chocolate,144,148,0,1,15,137
4,Amul Milk 1L,66,65,2,1,12,63


In [165]:

from sklearn.ensemble import RandomForestRegressor

In [166]:
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test)

In [167]:
r2_score(y_test,y_pred)*100

99.93175329054999

In [168]:
train_score = rf_model.score(X_train, y_train)  # R² score on training set
test_score = rf_model.score(X_test, y_test)  # R² score on test set

print(f"Training R² Score: {train_score:.4f}")
print(f"Testing R² Score: {test_score:.4f}")


Training R² Score: 0.9999
Testing R² Score: 0.9993


In [169]:
res=rf_model.predict([[125,126,2,1,18]])
res

d:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([485.295])

In [170]:
l=[]
for i in range(1,200):
    rf=RandomForestRegressor(n_estimators=i,random_state=42)
    rf.fit(X_train,y_train)
    y_pred=rf.predict(X_test)
    l.append(r2_score(y_test,y_pred)*100)



In [171]:
max(l)

99.93246344195582

In [ ]:
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.metrics import mean_absolute_error, r2_score

# # Load dataset
# df = pd.read_csv("blinkit_dynamic_pricing_1000_samples.csv")

# # Clean price columns by removing the "₹" symbol and converting to float
# df["Base Price"] = df["Base Price"].replace({'₹': ''}, regex=True).astype(float)
# df["Competitor Price"] = df["Competitor Price"].replace({'₹': ''}, regex=True).astype(float)
# df["Optimized Price"] = df["Optimized Price"].replace({'₹': ''}, regex=True).astype(float)

# # Encode categorical variables (Demand, Supply)
# # encoder = LabelEncoder()
# # df["Demand"] = encoder.fit_transform(df["Demand"])   # e.g., High -> 2, Medium -> 1, Low -> 0
# # df["Supply"] = encoder.fit_transform(df["Supply"])

# # Define features (X) and target (y)
# X = df[["Base Price", "Competitor Price", "Demand", "Supply", "Expiry Days"]]
# y = df["Optimized Price"]

# # Normalize numerical features using StandardScaler
# scaler = StandardScaler()
# X[["Base Price", "Competitor Price", "Expiry Days"]] = scaler.fit_transform(X[["Base Price", "Competitor Price", "Expiry Days"]])

# # Split dataset into training and testing sets (80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize and train the XGBoost Regressor
# xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
# xgb_model.fit(X_train, y_train)

# # Predict on the test set
# y_pred = xgb_model.predict(X_test)

# # Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred) * 100

# print(f"Mean Absolute Error (MAE): {mae:.2f}")
# print(f"R² Score: {r2:.4f}")

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

# Load dataset
df = pd.read_csv("blinkit_dynamic_pricing_1000_samples.csv")

# Clean price columns by removing the "₹" symbol and converting to float
df["Base Price"] = df["Base Price"].replace({'₹': ''}, regex=True).astype(float)
df["Competitor Price"] = df["Competitor Price"].replace({'₹': ''}, regex=True).astype(float)
df["Optimized Price"] = df["Optimized Price"].replace({'₹': ''}, regex=True).astype(float)

# For this example, we assume Demand and Supply columns are already numeric.
# If they are categorical, you would need to encode them.

# Define features (X) and target (y)
X = df[["Base Price", "Competitor Price", "Demand", "Supply", "Expiry Days"]]
y = df["Optimized Price"]

# Normalize numerical features using StandardScaler
scaler = StandardScaler()
numerical_cols = ["Base Price", "Competitor Price", "Expiry Days"]
X.loc[:, numerical_cols] = scaler.fit_transform(X.loc[:, numerical_cols])

# Optionally, save the scaler for future predictions
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost Regressor
xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred) * 100

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.4f}")

# Save the trained model to a pickle file for later use
with open("xgb_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)

print("✅ Model and scaler have been saved!")


C:\Users\ssona\AppData\Local\Temp\ipykernel_17364\1083164507.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[["Base Price", "Competitor Price", "Expiry Days"]] = scaler.fit_transform(X[["Base Price", "Competitor Price", "Expiry Days"]])


Mean Absolute Error (MAE): 1.49
R² Score: 99.9761


In [180]:
import pandas as pd
import numpy as np

# Example new input data (replace these values with your actual new sample)
new_sample = {
    "Base Price": [27],
    "Competitor Price": [30],
    "Demand": [1],      # Categorical input
    "Supply": [2],    # Categorical input
    "Expiry Days": [8]
}
new_df = pd.DataFrame(new_sample)

# Assume that 'encoder' and 'scaler' are already fitted on your training data.
# For example:
# encoder = LabelEncoder() and then fitted on the 'Demand' and 'Supply' columns in your training set.
# scaler = StandardScaler() and then fitted on the numeric columns.

# Encode categorical variables
new_df["Demand"] = encoder.transform(new_df["Demand"])
new_df["Supply"] = encoder.transform(new_df["Supply"])

# Scale numerical features
new_df[["Base Price", "Competitor Price", "Expiry Days"]] = scaler.transform(
    new_df[["Base Price", "Competitor Price", "Expiry Days"]]
)

# Predict using your trained XGBoost model (xgb_model)
predicted_price = xgb_model.predict(new_df)[0]
print(f"Predicted Optimized Price: {predicted_price:.2f}")


Predicted Optimized Price: 24.95


In [177]:
import pickle

In [178]:
with open("xgb_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)